### Load the dependencies

In [1]:
# Import the keras libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout
from keras.layers.embeddings import Embedding #To convert an integer to embedding
from keras.preprocessing import sequence #To convert a variable length sentence into a prespecified length
# fix random seed for reproducibility
np.random.seed(7)
from keras.utils.np_utils import to_categorical
import pandas as pd

C:\Users\SUNEEL\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Import the sklearn libraries
import re
import sklearn
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB

In [3]:
# Check the relevant path in your local machine
import os
os.getcwd()

'C:\\Users\\SUNEEL'

In [44]:
# Read the data

sms_data = pd.read_csv('TRAIN_SMS.csv')
print(sms_data)

      Label                                            Message
0       ham                 oh how abt 2 days before Christmas
1      info  Welcome to OVATION HOLD R.No. 184, 114, 395, 3...
2      info  Thank you for using your ICICI bank CREDITcard...
3       ham  schedule a meeting with the entire team in the...
4       ham                                Tommy is my brother
5      spam  OTP is 817453 for the txn of INR 8262.00 at SP...
6       ham                   the meeting is scheduled by john
7      spam  Dear customer, We wish you a Merry Christmas. ...
8      spam  Delivered: Your package withPawzone Red 1.25 i...
9      info  The PNR for your Air India Flt 7I115 for PGH-B...
10     info  Bimal Auto Agency : Service of your car KA52C8...
11     info  Appointment with Dr Clayton in Pune on 2011-08...
12     info  Maha Veer Auto Agency : Service of your car KA...
13     spam  Dear AirAsia Customer, flight 5Q658 from RJA s...
14     info  Dear Guest, Thanks for choosing Forlini's 

In [45]:
# Simple definition to process the words further

def message_to_words(raw_message):
#     letters_only = re.sub("[^a-zA-Z]", " ", raw_converse) 
    # 1. Lower case & split  
    words = raw_message.lower().split()                             
    # 2. Convert stop words to a set
    stops = set(stopwords.words("english"))                  
    # 3. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # 4. Join the words back into one string
    return(b" ".join(meaningful_words)) 

In [6]:
# Checking the length of the column
num_message = sms_data["Message"].size

In [11]:
print(num_message)
#print(num_message[:3], Label[:3])

30000


In [17]:
# Initialize empty directories

Message = []
Label = [] # First target

for x in range(len(sms_data.Message)):
    Message.append(message_to_words(str(sms_data.Message[x]).encode('ascii','ignore')))
    Label.append(sms_data.Label[x])

### Load the data

In [18]:
#Create word frequency matrix for every Message, 
#Apply Tfidf vectorizer-extracting upto 1400 features


vectorizer = CountVectorizer(analyzer = "word",\
                             tokenizer = None,\
                             preprocessor = None,
                             ngram_range=(1,1), # We are only interested in uni grams
                             max_features=1400, # Limits to 1400 features
                             stop_words = None) 

data_features = vectorizer.fit_transform(Message)
type(data_features)
vectorizer.get_feature_names()

['00',
 '000',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '10am',
 '10hrs',
 '10th',
 '11',
 '11am',
 '11hrs',
 '12',
 '121',
 '12hrs',
 '13',
 '13hrs',
 '14',
 '14hrs',
 '14th',
 '15',
 '15hrs',
 '15th',
 '16',
 '16hrs',
 '17',
 '17hrs',
 '17th',
 '18',
 '1800',
 '18hrs',
 '18th',
 '19',
 '19hrs',
 '19th',
 '1st',
 '20',
 '200',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '20hrs',
 '20th',
 '21',
 '21hrs',
 '22',
 '22hrs',
 '23',
 '23hrs',
 '24',
 '24hrs',
 '25',
 '250',
 '25hrs',
 '26',
 '26hrs',
 '27',
 '27hrs',
 '28',
 '28hrs',
 '29',
 '29hrs',
 '2mrw',
 '2nd',
 '2pm',
 '30',
 '300',
 '30am',
 '30hrs',
 '30pm',
 '31',
 '31hrs',
 '31st',
 '32',
 '32hrs',
 '33',
 '33hrs',
 '34',
 '34hrs',
 '35',
 '35hrs',
 '36',
 '36hrs',
 '37',
 '37hrs',
 '38',
 '38hrs',
 '39',
 '39hrs',
 '3g',
 '3pm',
 '3rd',
 '40',
 '40hrs',
 '41',
 '41hrs',
 '42',
 '42hrs',
 '43',
 '43hrs',
 '44',
 '44hrs',
 '45',
 '45hrs',
 '46',


In [19]:
print(len(Message), len(Label)) #30000 obs
print(np.unique(Label)) # 3 different categories

30000 30000
['ham' 'info' 'spam']


In [20]:
# Convert the data_features to array

# data features for categories
data_features = data_features.toarray()
data_features = pd.DataFrame(data_features)
data_features["Label"] = sms_data["Label"]
data_features = data_features.sample(frac =1)

# #data features for sub_categories

# data_features["sub_categories"] = health_data["sub_categories"]
# data_features = data_features.sample(frac =1)

In [21]:
# Verify the shape and head of data_features
print(data_features[:3])
print(data_features.shape)

       0  1  2  3  4  5  6  7  8  9  ...    1391  1392  1393  1394  1395  \
1252   0  0  0  0  0  0  0  0  0  0  ...       0     0     0     1     0   
10444  0  0  0  0  0  0  0  0  0  0  ...       0     0     0     0     0   
8994   1  0  0  0  0  0  0  0  0  0  ...       0     0     0     0     1   

       1396  1397  1398  1399  Label  
1252      0     0     0     0   spam  
10444     0     0     0     0    ham  
8994      0     0     0     0   spam  

[3 rows x 1401 columns]
(30000, 1401)


In [22]:
print(type(sms_data))
print(type(data_features))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [24]:
# Split into 60% train, 20% val and 20% test

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

#First split
train, validate, test = train_validate_test_split(data_features) 
cols = [col for col in data_features.columns if col not in ["Label"]]

train.x = train[cols]
train.y = train["Label"]
# train.z = train["sub_categories"]

validate.x = validate[cols]
validate.y = validate["Label"]
# validate.z = validate["sub_categories"]

test.x = test[cols]
test.y = test["Label"]
# test.z = test["sub_categories"]

C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:26

In [25]:
# Verify the shape of train.x, train.y
print(train.x.shape, train.y.shape)
print(validate.x.shape, validate.y.shape)
print(test.x.shape, test.y.shape)
# print(health_data.converse.map(len).max())

print(train.x[:3])
print(train.y[:3])

(18000, 1400) (18000,)
(6000, 1400) (6000,)
(6000, 1400) (6000,)
       0     1     2     3     4     5     6     7     8     9     ...   1390  \
23765     0     0     0     0     0     0     0     0     0     0  ...      0   
1035      0     0     1     0     0     0     0     0     0     0  ...      0   
24968     0     0     1     0     0     0     0     0     0     0  ...      0   

       1391  1392  1393  1394  1395  1396  1397  1398  1399  
23765     0     0     0     0     0     2     0     0     0  
1035      0     0     0     0     1     0     0     0     0  
24968     0     0     0     0     1     0     0     0     0  

[3 rows x 1400 columns]
23765    info
1035     info
24968    info
Name: Label, dtype: object


In [26]:
# Truncate and Pad input sequences

max_review_length = 500 # This should be actually health_data.converse.map(len).max()
train.x = sequence.pad_sequences(train.x, maxlen=max_review_length)
test.x = sequence.pad_sequences(test.x, maxlen=max_review_length)

ValueError: `sequences` must be a list of iterables. Found non-iterable: 0

### Let's inspect the first two sentences of train and their classes

In [27]:
# Simple Naive Bayes Model

Mnb = MultinomialNB()
Mnb.fit(train.x, train.y)
preds_NB = Mnb.predict(test.x)
confusion_matrix(test.y,preds_NB)

array([[1905,   27,   24],
       [   0, 2670,    0],
       [  31,    9, 1334]], dtype=int64)

In [28]:
# Naive Bayes Accuracy and recall

accuracy = metrics.accuracy_score(test.y,preds_NB)
recall = metrics.recall_score(test.y,preds_NB, average = 'macro')
print(accuracy, recall)

0.9848333333333333 0.9816047662847566


### Inspect the same sentences after padding

In [29]:
# Basic checks so that train.x, train.y,..,test.y are all in the same shape
print(np.unique(train.y))
# print(np.unique(train.z))

['ham' 'info' 'spam']


In [30]:
# Convert the train.y into simple categorical variables 0,1,2
train.y = pd.Categorical(train.y)
train.y.codes
train.y = to_categorical(np.asarray(train.y.codes))

# Similarly convert the test.y into simple categorical variables 0,1,2
test.y = pd.Categorical(test.y)
test.y.codes
test.y = to_categorical(np.asarray(test.y.codes))

# Similarly convert the val.y into simple categorical variables 0,1,2
validate.y = pd.Categorical(validate.y)
validate.y.codes
validate.y = to_categorical(np.asarray(validate.y.codes))

print(train.y.shape)

(18000, 3)


In [31]:
print(train.y.shape)
# print(train.z[:3])

(18000, 3)


In [37]:
# Convert the train.y into simple categorical variables from 0,1,2,..,6
train.z = pd.Categorical(train)
train.z.codes
train.z = to_categorical(np.asarray(train.z.codes))

# Similarly convert the test.y into simple categorical variables from 0,1,2,..,6
test.z = pd.Categorical(test)
test.z.codes
test.z = to_categorical(np.asarray(test.z.codes))

# Similarly convert the val.y into simple categorical variables from 0,1,2,..,6
validate.z = pd.Categorical(validate)
validate.z.codes
validate.z = to_categorical(np.asarray(validate.z.codes))

C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':


In [38]:
# Verify the shape of train.x and train.y before feeding into the model
print(train.x.shape, train.y.shape)
print(type(train.x), type(train.y))

print(validate.x.shape, validate.y.shape)

(18000, 1400) (18000, 3)
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
(6000, 1400) (6000, 3)


In [39]:
# Creating a custome "Recall" error function in Keras backend

import keras.backend as K

def recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (PP + K.epsilon())
    return recall

In [40]:
# Simple MLP, SGD and Dropout [6 categories]

from keras.optimizers import SGD

model = Sequential()
model.add(Dense(128, input_dim=1400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall])

# sgd = SGD(lr=0.04, decay=1e-6, momentum=0.6, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) 0.82% test accuracy
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               179328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 179,715
Trainable params: 179,715
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
# Fit the model for y = categories
model.fit(np.array(train.x), train.y, nb_epoch=15, batch_size=32, verbose=2, validation_data=(np.array(validate.x), validate.y))

scores_mlp = model.evaluate(np.array(test.x), test.y, verbose=0)
print("Accurcay_mlp_cat: %.2f%%" % (scores_mlp[1]*100))
print("Recall_mlp_cat: %.2f%%" % (scores_mlp[2]*100))

C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 18000 samples, validate on 6000 samples
Epoch 1/15
 - 6s - loss: 0.0933 - acc: 0.9738 - recall: 0.9500 - val_loss: 0.0146 - val_acc: 0.9975 - val_recall: 0.9973
Epoch 2/15
 - 3s - loss: 0.0093 - acc: 0.9979 - recall: 0.9979 - val_loss: 0.0113 - val_acc: 0.9977 - val_recall: 0.9973
Epoch 3/15
 - 3s - loss: 0.0049 - acc: 0.9986 - recall: 0.9986 - val_loss: 0.0113 - val_acc: 0.9970 - val_recall: 0.9970
Epoch 4/15
 - 3s - loss: 0.0027 - acc: 0.9993 - recall: 0.9993 - val_loss: 0.0110 - val_acc: 0.9972 - val_recall: 0.9972
Epoch 5/15
 - 3s - loss: 0.0019 - acc: 0.9996 - recall: 0.9996 - val_loss: 0.0114 - val_acc: 0.9972 - val_recall: 0.9972
Epoch 6/15
 - 3s - loss: 0.0020 - acc: 0.9996 - recall: 0.9996 - val_loss: 0.0109 - val_acc: 0.9975 - val_recall: 0.9975
Epoch 7/15
 - 3s - loss: 0.0016 - acc: 0.9997 - recall: 0.9997 - val_loss: 0.0113 - val_acc: 0.9977 - val_recall: 0.9977
Epoch 8/15
 - 3s - loss: 0.0014 - acc: 0.9998 - recall: 0.9998 - val_loss: 0.0128 - val_acc: 0.9973 - va

In [47]:
# Simple MLP, SGD and Dropout [21 sub_categories]

model = Sequential()
model.add(Dense(128, input_dim=5000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall])

# sgd = SGD(lr=0.04, decay=1e-6, momentum=0.6, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy', recall])
print(model.summary())

# Fit the model for y = categories
model.fit(np.array(train.x), train.y, nb_epoch=20, batch_size=32, verbose=2, validation_data=(np.array(validate.x), validate.y))

scores_mlp = model.evaluate(np.array(test.x), test.z, verbose=0)
print("Accurcay_mlp_sub_cat: %.2f%%" % (scores_mlp[1]*100))
print("Recall_mlp_sub_cat: %.2f%%" % (scores_mlp[2]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               640128    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 21)                2709      
Total params: 642,837
Trainable params: 642,837
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


ValueError: Error when checking input: expected dense_9_input to have shape (5000,) but got array with shape (1400,)

In [48]:
# LSTM Model [3 categories]

top_words = 500
embedding_vecor_length = 32
max_converse_length = 500
# max_converse_length = health_data.converse.map(len).max() # This gives the max length of the converse, used 500 here
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=1400))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall])
print(model.summary())

model.fit(np.array(train.x), train.y, nb_epoch=10, batch_size=64, validation_data=(np.array(validate.x), validate.y))
# Final evaluation of the model
scores = model.evaluate(np.array(test.x), test.y, verbose=0)
print("Accuracy_lstm_cat: %.2f%%" % (scores[1]*100))
print("Recall_lstm_cat: %.2f%%" % (scores[2]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1400, 32)          16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 303       
Total params: 69,503
Trainable params: 69,503
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\SUNEEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 18000 samples, validate on 6000 samples
Epoch 1/10
18000/18000 [==============================] - 1043s 58ms/step - loss: 0.9424 - acc: 0.5866 - recall: 0.5036 - val_loss: 0.7611 - val_acc: 0.7038 - val_recall: 0.2300
Epoch 2/10
18000/18000 [==============================] - 957s 53ms/step - loss: 0.6728 - acc: 0.7342 - recall: 0.3946 - val_loss: 0.6571 - val_acc: 0.7425 - val_recall: 0.6425
Epoch 3/10
18000/18000 [==============================] - 1026s 57ms/step - loss: 0.5919 - acc: 0.7721 - recall: 0.4084 - val_loss: 0.5501 - val_acc: 0.7928 - val_recall: 0.4838
Epoch 4/10
18000/18000 [==============================] - 1024s 57ms/step - loss: 0.5029 - acc: 0.8143 - recall: 0.6405 - val_loss: 0.4344 - val_acc: 0.8213 - val_recall: 0.5463
Epoch 5/10
18000/18000 [==============================] - 1119s 62ms/step - loss: 0.4325 - acc: 0.8287 - recall: 0.6659 - val_loss: 0.4178 - val_acc: 0.8278 - val_recall: 0.6208
Epoch 6/10
18000/18000 [==============================] - 1226

In [0]:
# # LSTM Model [21 categories]

# top_words = 500
# embedding_vecor_length = 32
# max_converse_length = 500
# # max_converse_length = health_data.converse.map(len).max() # This gives the max length of the converse, used 500 here
# model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=500))
# model.add(LSTM(100))
# model.add(Dense(21, activation='sigmoid'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall])
# print(model.summary())

# model.fit(np.array(train.x), train.z, nb_epoch=1, batch_size=64, validation_data=(np.array(val.x), val.z))
# # Final evaluation of the model
# scores = model.evaluate(np.array(test.x), test.z, verbose=0)
# print("Accuracy_lstm_sub_cat: %.2f%%" % (scores[1]*100))
# print("Recall_lstm_sub_cat: %.2f%%" % (scores[2]*100))

In [0]:
# CNN 1D 

# set parameters:
max_features = 500
maxlen = 1400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10 #Models computational time is a lot

In [0]:
# CNN 1D [3 categories]

from keras.layers import Conv1D, GlobalMaxPooling1D

print('Build Convolution 1D model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.3))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 border_mode='valid',
                 activation='relu',
                 subsample_length=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall])

Build Convolution 1D model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1400, 50)          25000     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1400, 50)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1398, 250)         37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_3 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)      

/nfsroot/data/home/s_jaysetty/.local/lib/python2.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(250, 3, padding="valid", strides=1, activation="relu")`


In [0]:
model.fit(np.array(train.x), train.y, batch_size=batch_size, nb_epoch=epochs, validation_data=(np.array(validate.x), validate.y) )

# Final evaluation of the model
scores = model.evaluate(np.array(test.x), test.y, verbose=0)
print("Accuracy_cnn1d_cat: %.2f%%" % (scores[1]*100))
print("Recall_cnn1d_cat: %.2f%%" % (scores[2]*100))

Train on 18000 samples, validate on 6000 samples
Epoch 1/10
18000/18000 [==============================] - 8s - loss: 0.5914 - acc: 0.7303 - recall: 0.1734 - val_loss: 1.3650 - val_acc: 0.4250 - val_recall: 0.3150
Epoch 2/10
18000/18000 [==============================] - 7s - loss: 0.4518 - acc: 0.8038 - recall: 0.1626 - val_loss: 2.2774 - val_acc: 0.3662 - val_recall: 0.3178
Epoch 3/10
18000/18000 [==============================] - 7s - loss: 0.4269 - acc: 0.8149 - recall: 0.1879 - val_loss: 3.1005 - val_acc: 0.3387 - val_recall: 0.3262
Epoch 4/10
18000/18000 [==============================] - 6s - loss: 0.4132 - acc: 0.8196 - recall: 0.2393 - val_loss: 4.3311 - val_acc: 0.3315 - val_recall: 0.3262
Epoch 5/10
18000/18000 [==============================] - 6s - loss: 0.4065 - acc: 0.8206 - recall: 0.2596 - val_loss: 5.0334 - val_acc: 0.3347 - val_recall: 0.3262
Epoch 6/10
18000/18000 [==============================] - 6s - loss: 0.3984 - acc: 0.8266 - recall: 0.2358 - val_loss: 4.9216 

In [0]:
# # CNN 1D [21 categories]

# from keras.layers import Conv1D, GlobalMaxPooling1D

# print('Build Convolution 1D model...')
# model = Sequential()

# # we start off with an efficient embedding layer which maps
# # our vocab indices into embedding_dims dimensions
# model.add(Embedding(max_features,
#                     embedding_dims,
#                     input_length=maxlen))
# model.add(Dropout(0.2))

# # we add a Convolution1D, which will learn filters
# # word group filters of size filter_length:
# model.add(Conv1D(filters,
#                  kernel_size,
#                  border_mode='valid',
#                  activation='relu',
#                  subsample_length=1))
# # we use max pooling:
# model.add(GlobalMaxPooling1D())

# # We add a vanilla hidden layer:
# model.add(Dense(hidden_dims))
# model.add(Dropout(0.2))
# model.add(Activation('relu'))

# # We project onto a single unit output layer, and squash it with a sigmoid:
# model.add(Dense(21))
# model.add(Activation('sigmoid'))

# model.summary()

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall])

In [0]:
# model.fit(np.array(train.x), train.z, batch_size=batch_size, nb_epoch=epochs, validation_data=(np.array(val.x), val.z) )

# # Final evaluation of the model
# scores = model.evaluate(np.array(test.x), test.z, verbose=0)
# print("Accuracy_cnn1d_sub_cat: %.2f%%" % (scores[1]*100))
# print("Recall_cnn1d_sub_cat: %.2f%%" % (scores[2]*100))

In [0]:
# import matplotlib.pyplot as plt


# model_mlp = model.fit(np.array(train.x), train.z, nb_epoch=20, batch_size=32, verbose=2, validation_data=(np.array(val.x), val.z))


In [0]:
# history = model_mlp
# # list all data in history
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [0]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()